Восстановим константы, словарь и модели из прошлого нотубка

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from src.models.models import TransformerClassifier, LSTMClassifier, CustomMambaClassifier, SimpleMambaBlock
from src.data_utils.config import DatasetConfig
from src.data_utils.dataset_params import DatasetName
from src.data_utils.dataset_generator import DatasetGenerator

MAX_SEQ_LEN = 300
EMBEDDING_DIM = 128
BATCH_SIZE = 32 
NUM_CLASSES = 2
SAVE_DIR = "../pretrained_comparison" 
DATA_DIR = "../datasets" 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = DatasetConfig(load_from_disk=True, path_to_data=DATA_DIR)
generator = DatasetGenerator(DatasetName.IMDB, config=config)

_, _, _ = generator.generate_dataset() 
vocab = generator.vocab
VOCAB_SIZE = len(vocab)
text_processor = generator.get_text_processor()

Возьмем всопомгательную функцию из пролшло нотубка

In [2]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_on_test(model, test_loader, device, criterion):
    model.eval()
    total_test_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
            
    avg_test_loss = total_test_loss / len(test_loader)
        
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    
    return {'loss': avg_test_loss, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1}

Создадим генератор датасета и передадим в него уже готовый текстовый процессор, заберем датасет из другого распределения

In [3]:

def create_dataloader(X, y, batch_size, shuffle=True):
    X_tensor = torch.as_tensor(X, dtype=torch.long)
    y_tensor = torch.as_tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

generator_polarity = DatasetGenerator(DatasetName.POLARITY, config=config)
generator_polarity.text_processor = text_processor
(X_train, y_train), (X_val, y_val), (X_test, y_test) = generator_polarity.generate_dataset()


test_loader = create_dataloader(X_test, y_test, BATCH_SIZE, shuffle=False)

Восстановим конфигурации конфигов моделей из прошлого нотубка

In [4]:
model_configs = {
    "CustomMamba": {
        "class": CustomMambaClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'd_model': EMBEDDING_DIM, 'd_state': 8, 
                   'd_conv': 4, 'num_layers': 2, 'num_classes': NUM_CLASSES},
    },
    "Lib_LSTM": {
        "class": LSTMClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'embed_dim': EMBEDDING_DIM, 'hidden_dim': 128, 
                   'num_layers': 2, 'num_classes': NUM_CLASSES, 'dropout': 0.5},
    },
    "Lib_Transformer": {
        "class": TransformerClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'embed_dim': EMBEDDING_DIM, 'num_heads': 4, 
                   'num_layers': 2, 'num_classes': NUM_CLASSES, 'max_seq_len': MAX_SEQ_LEN},
    },
}

Теперь посмотрим на результаты

In [5]:
results = {}
for model_name, config in model_configs.items():        
    model_path = os.path.join(SAVE_DIR, f"best_model_{model_name.lower()}.pth")            
    model = config['class'](**config['params']).to(DEVICE)

    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    criterion = nn.CrossEntropyLoss()
    test_metrics = evaluate_on_test(model, test_loader, DEVICE, criterion)
    results[model_name] = test_metrics
    
results_df = pd.DataFrame(results).T
print("\n\n--- Итоговая таблица сравнения моделей на тестовых данных ---")
print(results_df.to_string())


/home/gab1k/.cache/pypoetry/virtualenvs/monkey-coding-dl-project-F4QJzkF_-py3.12/lib/python3.12/site-packages/torch/nn/modules/transformer.py:505: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(




--- Итоговая таблица сравнения моделей на тестовых данных ---
                     loss  accuracy  precision    recall  f1_score
CustomMamba      0.815722    0.5122   0.511772  0.814917  0.628711
Lib_LSTM         0.737277    0.5016   0.522581  0.191792  0.280600
Lib_Transformer  0.707197    0.5250   0.554118  0.321231  0.406695


Снимали тут на "игрушечных данных". На даже на них видно, что:
 - accuracy порядка 50% на всех моделях
    - Значит зависимости выучить не удалось
    - Но что интересно, recall и f1 у Mamba значительно выше, чем у LSTM и Transformer
    - Вероятнее всего дисбаланс классов + переобучение Mamba

Теперь попробуем снять на более "серьезных моделях", обученных в файле mamba_vs_transformerts.ipynb

In [6]:
SAVE_DIR = "../"  # положили модели на удаленной машине сюда
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = DatasetConfig(
    load_from_disk=True,
    path_to_data="../datasets",
    train_size=25000,  # увеличили количество сэмплов
    val_size=12500,
    test_size=12500
)
generator = DatasetGenerator(DatasetName.IMDB, config=config)

_, _, _ = generator.generate_dataset() 
vocab = generator.vocab
VOCAB_SIZE = len(vocab)
text_processor = generator.get_text_processor()
generator_polarity = DatasetGenerator(DatasetName.POLARITY, config=config)
generator_polarity.text_processor = text_processor
(X_train, y_train), (X_val, y_val), (X_test, y_test) = generator_polarity.generate_dataset()

test_loader = create_dataloader(X_test, y_test, BATCH_SIZE, shuffle=False)

In [7]:
model_configs = {
    "CustomMamba": {
        "class": CustomMambaClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'd_model': EMBEDDING_DIM, 'd_state': 8, 
                   'd_conv': 4, 'num_layers': 2, 'num_classes': NUM_CLASSES},
    },
    "Lib_Transformer": {
        "class": TransformerClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'embed_dim': EMBEDDING_DIM, 'num_heads': 8, 
                   'num_layers': 4, 'num_classes': NUM_CLASSES, 'max_seq_len': MAX_SEQ_LEN},
    },
}

In [8]:
results = {}
for model_name, config in model_configs.items():        
    model_path = os.path.join(SAVE_DIR, f"best_model_{model_name.lower()}.pth")            
    model = config['class'](**config['params']).to(DEVICE)

    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    criterion = nn.CrossEntropyLoss()
    test_metrics = evaluate_on_test(model, test_loader, DEVICE, criterion)
    results[model_name] = test_metrics
    
results_df = pd.DataFrame(results).T
print("\n\n--- Итоговая таблица сравнения моделей на тестовых данных ---")
print(results_df.to_string())




--- Итоговая таблица сравнения моделей на тестовых данных ---
                     loss  accuracy  precision    recall  f1_score
CustomMamba      1.454583   0.49048   0.482340  0.202358  0.285105
Lib_Transformer  0.820102   0.49120   0.495571  0.748725  0.596395


Гипотеза не подтвердилась: 
 - Если брать данные из разных семейств, то модель ведет себя непредсказуемо